In [80]:
import pandas as pd

In [81]:
df = pd.read_csv(r'D:\project\cv_final\data\Food Ingredients and Recipe Dataset with Image Name Mapping.csv')
df.head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [82]:
df = df.drop(columns=['Unnamed: 0'])
df.columns

Index(['Title', 'Ingredients', 'Instructions', 'Image_Name',
       'Cleaned_Ingredients'],
      dtype='object')

# Check null

In [110]:
df.isnull().sum()

Title                  5
Ingredients            0
Instructions           8
Image_Name             0
Cleaned_Ingredients    0
Image_Path             0
dtype: int64

In [ ]:
df = df.filter(df['Title'].isnotnull())
df.isnull().sum()   

AttributeError: 'Series' object has no attribute 'isnotnull'

# Ingredient parser

In [83]:

import re

# Map các kí tự phân số Unicode về dạng số thập phân hoặc bỏ luôn
UNICODE_FRACTIONS = {
    "¼": "1/4",
    "½": "1/2",
    "¾": "3/4",
    "⅐": "1/7",
    "⅑": "1/9",
    "⅒": "1/10",
    "⅓": "1/3",
    "⅔": "2/3",
    "⅕": "1/5",
    "⅖": "2/5",
    "⅗": "3/5",
    "⅘": "4/5",
    "⅙": "1/6",
    "⅚": "5/6",
    "⅛": "1/8",
    "⅜": "3/8",
    "⅝": "5/8",
    "⅞": "7/8",
}

UNITS = [
    "tsp", "tbsp", "cup", "cups", "kg", "g", "mg", "oz",
    "pound", "lb", "lbs", "teaspoon", "tablespoon",
    "clove", "cloves", "slice", "slices", "pinch", "pounds",
   "tablespoons", "teaspoons", "litre", "litres", "ml", "millilitre", "millilitres",
   "glass", "glasses", "can", "cans", "package", "packages", "stick", "sticks", "qt",
   "inch", "inches", "bunch", "bunches", "whole", "small", "large", "medium", "ounce", "ounces"
]

def replace_unicode_fractions(text: str):
    for uf, ascii_fr in UNICODE_FRACTIONS.items():
        text = text.replace(uf, ascii_fr)
    return text


def clean_quantity(text: str):
    """
    Remove amounts (including unicode fractions, ASCII fractions, mixed numbers)
    Examples handled:
      3½ → remove
      1/4 → remove
      3 1/2 → remove
      2 ¾ → remove
    """
    text = text.lower()
    text = replace_unicode_fractions(text)
    text = text.replace("-", " ").replace("–", " ")
    # Remove patterns like: "3 1/2", "1/4"
    text = re.sub(r"\b\d+\s*\d*\/\d+\b", "", text)

    # Remove any remaining pure numbers
    text = re.sub(r"\b\d+\b", "", text)

    # Remove units (tsp, cups, g, ml…)
    text = re.sub(r"\b(?:%s)\b" % "|".join(UNITS), "", text)

    # Remove punctuation
    text = re.sub(r"[()\",.]", " ", text)
    text = re.sub(r"[^\w\s]", " ", text)  
    text = text.replace("'", " ")
    # Clean spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


def extract_head_ingredient(ingredient: str):
    COOKING_WORDS = [
        "finely", "chopped", "fresh", "freshly", "ground", "small",
        "large", "extra", "virgin", "unsalted", "divided", "plus",
        "room", "temperature", "thinly", "sliced", "melted", "cut", "diced",
        "divided", "to", "taste", "removed", "minced", "peeled", "ripe", "coarsely",
        "grated", "boiled", "roasted", "peeled", "blanched", "crushed", "minced",
        "into"
    ]

    ing = clean_quantity(ingredient)
    tokens = [t for t in ing.split() if t not in COOKING_WORDS]
    out = " ".join(set(tokens))
    return out

In [84]:
sample1 = df.iloc[4]["Cleaned_Ingredients"]
print(sample1)

['1 teaspoon dark brown sugar', '1 teaspoon hot water', '1 ½ oz. bourbon', '½ oz. fresh lemon juice', '2 teaspoons apple butter (storebought or homemade)', 'Garnish: orange twist and freshly grated or ground cinnamon']


In [85]:
extract_head_ingredient(sample1)


'hot butter water garnish cinnamon apple orange dark or homemade sugar bourbon twist brown lemon storebought and juice'

In [86]:
df["Cleaned_Ingredients"]= df["Cleaned_Ingredients"].apply(extract_head_ingredient)
df.head()

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,purpose bread onion oil total cored olive as r...
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,salt about in thyme kosher new pepper potatoes...
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,onion cheese sharp full elbow macaroni pepper ...
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,celery turkey heavy ribs cubes oil olive butte...
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,hot butter water garnish cinnamon apple orange...


In [ ]:
#

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Image_Path
11221,NaN,[],NaN,roasted-game-hens-with-caramelized-root-vegeta...,,data\Food Images\roasted-game-hens-with-carame...
12373,NaN,[],NaN,chicken-soup-with-rice-232605,,data\Food Images\chicken-soup-with-rice-232605...
12378,NaN,[],NaN,double-lemon-bars-232572,,data\Food Images\double-lemon-bars-232572.jpg
12818,NaN,[],NaN,pear-and-frangipane-crostata-with-raspberry-vi...,,data\Food Images\pear-and-frangipane-crostata-...
12829,NaN,[],NaN,hazelnut-shortbread-sticks-231311,,data\Food Images\hazelnut-shortbread-sticks-23...


In [ ]:
import os
image_folder = r"D:\project\cv_final\data\Food Images"

# Tạo path đúng
df["Image_Path"] = df["Image_Name"].apply(lambda x: os.path.join(image_folder, x + ".jpg"))

In [87]:
df.to_csv(r'D:\project\cv_final\data\Food Ingredients and Recipe Dataset with Cleaned Ingredients.csv', index=False)